In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder

import os

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
print(os.listdir('../input/'))

In [ ]:
app_train=pd.read_csv('../input/application_train.csv')
print(app_train.shape)
app_train.head()

This training data has 307511 observations, that each row is a separate loan and 122 features (variables). Label is TARGET

In [ ]:
app_test=pd.read_csv('../input/application_test.csv')
print(app_test.shape)
app_test.head()

This testing data has 48744 observations, which is considerably smaller and lacks TARGET column.

# Explortary Data Analysis

## Distribution of Target column

In [ ]:
app_train['TARGET'].value_counts()

In [ ]:
app_train['TARGET'].plot.hist()

From the distribution of the target column, this is a highly imbalanced classification problem.

## Examine Missing Values

In [ ]:
def missing_values(df):
    missing_count=df.isnull().sum()
    missing_count_percent=df.isnull().sum()/len(df)
    
    missing_df=pd.DataFrame(missing_count,columns=['missing count'])    
    missing_df['missing frequency']=list(missing_count_percent)
    missing_df['missing frequency']=missing_df['missing frequency'].apply(lambda x: round(x,2))
    missing_df=missing_df.loc[missing_df['missing count']!=0]
    
    missing_df.sort_values('missing frequency',ascending=False,inplace=True)
    
    print('There are %d features possess missing values'%len(missing_df))
    
    return missing_df

In [ ]:
mv_df=missing_values(app_train)

In [ ]:
display(mv_df.head(10))

## Column Types

In [ ]:
app_train.dtypes.value_counts()

In [ ]:
app_train.select_dtypes('object').apply(pd.Series.nunique, axis=0)

## Label coding and One-Hot coding

In [ ]:
le=LabelEncoder()
le_count=0

for col in app_train.columns:
    if app_train[col].dtypes=='object':
        if len(list(app_train[col].unique()))<=2:
            le.fit(app_train[col])
            app_train[col]=le.transform(app_train[col])
            app_test[col]=le.transform(app_test[col])
            
            le_count+=1
            
print('%d columns were label encoded.'%le_count)

In [ ]:
app_train=pd.get_dummies(app_train)
app_test=pd.get_dummies(app_test)

print('Trainning Feature shape:',app_train.shape)
print('Testing Feature shape:',app_test.shape)

## Aligning Training and Testing Data

In [ ]:
train_labels=app_train['TARGET'].copy()
app_train,app_test=app_train.align(app_test, join='inner',axis=1)

app_train['TARGET']=train_labels
print('Trainning Feature shape:',app_train.shape)
print('Testing Feature shape:',app_test.shape)

## Back to Exploratory Data Analysis

In [ ]:
(app_train['DAYS_BIRTH']/-365).describe()

In [ ]:
(app_train['DAYS_EMPLOYED']).describe()

In [ ]:
app_train['DAYS_EMPLOYED'].plot.hist(title='Days Employment Histogram')
plt.xlabel('Days Employment')

Is anomalous person has more default?

In [ ]:
anom=app_train[app_train['DAYS_EMPLOYED']==365243]
non_anom=app_train[app_train['DAYS_EMPLOYED']!=365243]

print('Rate of default for anomalous: %0.2f%%'%(100*anom['TARGET'].mean()))

print('Rate of default for non-anomalous: %0.2f%%'%(100*non_anom['TARGET'].mean()))

print('Total number of days employed anomalous person: %d'%len(anom))

In [ ]:
app_train['DAYS_EMPLOYED_ANOM']=app_train['DAYS_EMPLOYED']==365243
app_train['DAYS_EMPLOYED'].replace({365243:np.nan},inplace=True)
app_train['DAYS_EMPLOYED'].plot.hist(title='Days employed')

In [ ]:
app_test['DAYS_EMPLOYED_ANOM']=app_test['DAYS_EMPLOYED']==365243
app_test['DAYS_EMPLOYED'].replace({365243:np.nan},inplace=True)

print('Threre are %d anomous in %d test samples'%(app_test['DAYS_EMPLOYED_ANOM'].sum(),len(app_test)))

In [ ]:
correlation=app_train.corr()['TARGET'].sort_values()


In [ ]:
print('Most positively correlated:\n',correlation.tail(15))
print('Most negatively correlated:\n',correlation.head(15))

## Effect of age in repayment

In [ ]:
app_train['DAYS_BIRTH']=abs(app_train['DAYS_BIRTH'])

app_train['DAYS_BIRTH'].corr(app_train['TARGET'])

In [ ]:
plt.style.use('fivethirtyeight')

plt.hist(app_train['DAYS_BIRTH']/365.,edgecolor='k',bins=25)
plt.title('Age distribution')
plt.xlabel('Age')
plt.ylabel('Number')

In [ ]:
plt.figure(figsize=(10,7))

sns.kdeplot(app_train.loc[app_train['TARGET']==0,'DAYS_BIRTH']/365,label='target=0')
sns.kdeplot(app_train.loc[app_train['TARGET']==1,'DAYS_BIRTH']/365,label='target=1')


plt.title('KDE of age')
plt.xlabel('Age')
plt.ylabel('kde')

In [ ]:
age_data=app_train[['DAYS_BIRTH','TARGET']].copy()
age_data['YEAR_BIRTH']=age_data['DAYS_BIRTH']/365
age_data['YEARS_BINNED']=pd.cut(age_data['YEAR_BIRTH'],bins=np.linspace(20,70,num=11))

age_data.head(10)

In [ ]:
age_groups=age_data.groupby('YEARS_BINNED').mean()
age_groups

In [ ]:
plt.figure(figsize=(8,8))

plt.bar(age_groups.index.astype(str),100*age_groups['TARGET'])

plt.xticks(rotation=75)
plt.xlabel('Age Group (years)')
plt.ylabel('Failure to Repay (%)')
plt.title('Failure to Repay by Age Group')

## Exterior data source

In [ ]:
ext_data=app_train[['TARGET','EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']]

ext_data_corrs=ext_data.corr()
ext_data_corrs

In [ ]:
plt.figure(figsize=(8,6))

sns.heatmap(ext_data_corrs, cmap=plt.cm.RdYlBu_r,vmin=-0.15,annot=True,vmax=0.60)

In [ ]:
plt.figure(figsize=(10,12))
for i,term in enumerate(['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3']):
    plt.subplot(3,1,i+1)
    sns.kdeplot(app_train.loc[app_train['TARGET']==0,term],label='target==0')
    sns.kdeplot(app_train.loc[app_train['TARGET']==1,term],label='target==1')
    
    plt.title('Distribution of %s by Target value'%term)
    plt.xlabel('%s'%term)
    plt.ylabel('Density')
    
plt.tight_layout(h_pad=2.5)

In [ ]:
# Copy the data for plotting
plot_data = ext_data.drop(columns = ['DAYS_BIRTH']).copy()

# Add in the age of the client in years
plot_data['YEAR_BIRTH'] = age_data['YEAR_BIRTH']

# Drop na values and limit to first 100000 rows
plot_data = plot_data.dropna().loc[:100000, :]

# Function to calculate correlation coefficient between two columns
def corr_func(x, y, **kwargs):
    r = np.corrcoef(x, y)[0][1]
    ax = plt.gca()
    ax.annotate("r = {:.2f}".format(r),
                xy=(.2, .8), xycoords=ax.transAxes,
                size = 20)

# Create the pairgrid object
grid = sns.PairGrid(data = plot_data, size = 3, diag_sharey=False,
                    hue = 'TARGET', 
                    vars = [x for x in list(plot_data.columns) if x != 'TARGET'])

# Upper is a scatter plot
grid.map_upper(plt.scatter, alpha = 0.2)

# Diagonal is a histogram
grid.map_diag(sns.kdeplot)

# Bottom is density plot
grid.map_lower(sns.kdeplot, cmap = plt.cm.OrRd_r);

plt.suptitle('Ext Source and Age Features Pairs Plot', size = 32, y = 1.05);

## Feature engineering

### Polynomial Features

In [ ]:
poly_features=app_train[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']].copy()
poly_features_test=app_train[['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']].copy()

from sklearn.preprocessing import Imputer
imputer=Imputer(strategy='median')

poly_features=imputer.fit_transform(poly_features)

poly_features_test=imputer.transform(poly_features_test)

from sklearn.preprocessing import PolynomialFeatures

poly_transformer=PolynomialFeatures(degree=3)



In [ ]:

# Train the polynomial features
poly_transformer.fit(poly_features)

# Transform the features
poly_features = poly_transformer.transform(poly_features)
poly_features_test = poly_transformer.transform(poly_features_test)
print('Polynomial Features shape: ', poly_features.shape)


In [ ]:
poly_transformer.get_feature_names(input_features=['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH'])[:15]

In [ ]:
poly_features=pd.DataFrame(poly_features,columns=poly_transformer.get_feature_names(input_features=['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']))

poly_features['TARGET']=app_train['TARGET'].copy()

poly_corr=poly_features.corr()['TARGET'].sort_values()

print(poly_corr.head(10))
print(poly_corr.tail(10))

In [ ]:
poly_features_test=pd.DataFrame(poly_features_test,columns=poly_transformer.get_feature_names(input_features=['EXT_SOURCE_1','EXT_SOURCE_2','EXT_SOURCE_3','DAYS_BIRTH']))

poly_features['SK_ID_CURR']=app_train['SK_ID_CURR'].copy()

app_train_poly=app_train.merge(poly_features, how='left',on='SK_ID_CURR')

poly_features_test['SK_ID_CURR']=app_test['SK_ID_CURR'].copy()

app_test_poly=app_test.merge(poly_features_test, how='left',on='SK_ID_CURR')

app_train_poly,app_test_poly=app_train_poly.align(app_test_poly,join='inner',axis=1)

print('Training data with polynomial features shape: ',app_train_poly.shape)
print('Testing data with polynomial features shape: ',app_test_poly.shape)


## Domain Knowledge Features

In [ ]:
app_train_domain=app_train.copy()
app_test_domain=app_test.copy()

In [ ]:
app_train_domain['CREDIT_INCOME_PERCENT']=app_train_domain['AMT_CREDIT']/app_train_domain['AMT_INCOME_TOTAL']

app_train_domain['ANNUITY_INCOME_PERCENT']=app_train_domain['AMT_ANNUITY']/app_train_domain['AMT_INCOME_TOTAL']

app_train_domain['CREDIT_TERM']=app_train_domain['AMT_ANNUITY']/app_train_domain['AMT_CREDIT']

app_train_domain['DAYS_EMPLOYED_PERCENT']=app_train_domain['DAYS_EMPLOYED']/app_train_domain['DAYS_BIRTH']



In [ ]:
app_test_domain['CREDIT_INCOME_PERCENT']=app_test_domain['AMT_CREDIT']/app_test_domain['AMT_INCOME_TOTAL']

app_test_domain['ANNUITY_INCOME_PERCENT']=app_test_domain['AMT_ANNUITY']/app_test_domain['AMT_INCOME_TOTAL']

app_test_domain['CREDIT_TERM']=app_test_domain['AMT_ANNUITY']/app_test_domain['AMT_CREDIT']

app_test_domain['DAYS_EMPLOYED_PERCENT']=app_test_domain['DAYS_EMPLOYED']/app_test_domain['DAYS_BIRTH']


In [ ]:
plt.figure(figsize=(12,20))

for i,feature in enumerate(['CREDIT_INCOME_PERCENT','ANNUITY_INCOME_PERCENT','CREDIT_TERM','DAYS_EMPLOYED_PERCENT']):
    plt.subplot(4,1,i+1)
    
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET']==0,feature],label='target==0')
    sns.kdeplot(app_train_domain.loc[app_train_domain['TARGET']==1,feature],label='target==1')
    
    plt.title('Distribution of %s by Target Value'%feature)
    plt.xlabel('%s'%feature)
    plt.ylabel('Density')
    
plt.tight_layout(h_pad=2.5)

In [ ]:
from sklearn.preprocessing import MinMaxScaler,Imputer

if 'TARGET' in app_train:
    train=app_train.drop(columns=['TARGET'])
else:
    train=app_train.copy()
    
features=list(train.columns)

test=app_test.copy()

imputer=Imputer(strategy='median')

scaler=MinMaxScaler(feature_range=(0,1))

imputer.fit(train)

train=imputer.transform(train)

test=imputer.transform(test)

scaler.fit(train)

train=scaler.transform(train)

test=scaler.transform(test)

print('Training data shape',train.shape)
print('Testing data shape',test.shape)



In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg=LogisticRegression(C=0.0001)

log_reg.fit(train,train_labels)

In [ ]:
log_reg_pred=log_reg.predict_proba(test)[:,1]

In [ ]:
submit=app_test[['SK_ID_CURR']]
submit['TARGET']=log_reg_pred
submit.head()

In [ ]:
submit.to_csv('log_reg_baseline.csv',index=False)

## Improved Model: Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

random_forest=RandomForestClassifier(n_estimators=100,random_state=50,verbose=1,n_jobs=-1)

In [ ]:
random_forest.fit(train,train_labels)

feature_importance_values=random_forest.feature_importances_
feature_importances=pd.DataFrame({'feature':features,'importance': feature_importance_values})

predictions=random_forest.predict_proba(test)[:,1]

In [ ]:
submit=app_test['SK_ID_CURR']
submit['TARGET']=predictions

submit.to_csv('random_forest_baseline.csv',index=False)

## Make Predictions using Engineered Features

In [ ]:
poly_features_names=list(app_train_poly.columns)

imputer=Imputer(strategy='median')

poly_features=imputer.fit_transform(app_train_poly)
poly_features_test=imputer.transform(app_test_poly)

scaler=MinMaxScaler(feature_range=(0,1))

poly_features=scaler.fit_transform(poly_features)
poly_features_test=scaler.transform(poly_features_test)

random_forest_poly=RandomForestClassifier(n_estimators=100,random_state=50,verbose=1,n_jobs=-1)


In [ ]:
random_forest_poly.fit(poly_features,train_labels)

predictions=random_forest_poly.predict_proba(poly_features_test)[:,1]

In [ ]:
submit=app_test[['SK_ID_CURR']]

submit['TARGET']=predictions

submit.to_csv('random_forest_baseline_engineered.csv',index=False)

## Testing Domain Features

In [ ]:
app_train_domain=app_train_domain.drop(columns='TARGET')

domain_features_names=list(app_train_domain.columns)

domain_features=imputer.fit_transform(app_train_domain)

domain_features_test=imputer.transform(app_test_domain)

scaler=MinMaxScaler(feature_range=(0,1))

domain_features=scaler.fit_transform(domain_features)
domain_features_test=scaler.transform(domain_features_test)

random_forest_domain=RandomForestClassifier(n_estimators=100,random_state=50,verbose=1,n_jobs=-1)

random_forest_domain.fit(domain_features,train_labels)

feature_importances_values_domain=random_forest_domain.feature_importances_
feature_importances_domain=pd.DataFrame({'feature':domain_features_names,'importance':feature_importances_values_domain})

predictions=random_forest_domain.predict_proba(domain_features_test)[:,1]

In [ ]:
submit=app_test[['SK_ID_CURR']]
submit['TARGET']=predictions

submit.to_csv('random_forest_baseline_domain.csv',index=False)

## Feature importance

In [ ]:
def plot_feature_importances(df):
    df=df.sort_values('importance',ascending=False).reset_index()
    df['importance_normalized']=df['importance']/df['importance'].sum()
    
    plt.figure(figsize=(10,6))
    ax=plt.subplot()
    
    ax.barh(list(reversed(list(df.index[:15]))),df['importance_normalized'].head(15), align='center',edgecolor='k')
    
    ax.set_yticks(list(reversed(list(df.index[:15]))))
    ax.set_yticklabels(df['feature'].head(15))
    
    plt.xlabel('Normalized Importance')
    plt.title('Feature Importances')
    plt.show()
    
    return df

In [ ]:
feature_importances_sorted=plot_feature_importances(feature_importances)

In [ ]:
feature_importances_domain_sorted=plot_feature_importances(feature_importances_domain)